## Preprocessing and Feature Extraction

In [41]:
import snscrape.modules.twitter as sntwitter
import pandas as pd
from textblob import TextBlob 

from nltk.corpus import stopwords
from nltk.corpus import twitter_samples
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import nltk

from argparse import Namespace
from functools import partial
from pathlib import Path
from pprint import pprint

import os
import pickle
import random
import re
import string

In [42]:
df = pd.read_csv("../Data/Ukraine Tweets.csv")

## Sentiment Analysis:

Precounting of features: Length, Hashtags, URLs and Mentions

In [44]:
df['tweet_length'] = df['rendered_content'].apply(len)

In [58]:
df['num_mentions'] = df['rendered_content'].apply(lambda x: x.count('@'))

In [56]:
df['num_hashtags'] = df['rendered_content'].apply(lambda x: x.count('#'))

In [60]:
df['num_urls'] = df['rendered_content'].apply(lambda x: x.count('https'))

In [61]:
df

,id,date,user,user_followers,user_created,rendered_content,likes,retweets,replies,quoteCount,hashtags,lang,media,mentionedUsers,tweet_length,num_hashtags,num_mentions,num_urls
0,1477420789863436289,2022-01-01 23:25:40+00:00,anno1540,8838,2014-06-12 17:05:22+00:00,"Lithuania will never abandon Ukraine, voluntee...",5,1,0,0,"['Lithuania', 'Ukraine']",en,NaN,NaN,132,2,0,0
1,1477414596424220679,2022-01-01 23:01:03+00:00,weather_odessa,119,2019-07-10 08:34:22+00:00,#odessa #odesa #ukraine #одесса\nNow: 4.2°C\nT...,0,0,0,0,"['odessa', 'odesa', 'ukraine', 'одесса']",en,NaN,NaN,188,4,0,0
2,1477414332376010752,2022-01-01 23:00:00+00:00,AlArabiya_Eng,927174,2009-02-28 08:31:32+00:00,After tough talk between Presidents Joe Biden ...,4,0,3,0,"['Russia', 'Ukraine']",en,NaN,NaN,277,2,0,0
3,1477409748572151809,2022-01-01 22:41:47+00:00,beatravelling,6329,2014-02-28 21:25:33+00:00,The beach can be nice in the fall too 😊🇺🇦\n\n#...,0,0,0,0,"['lanzheron', 'langeron', 'beach', 'odessa', '...",en,NaN,NaN,122,5,0,0
4,1477409332820119552,2022-01-01 22:40:08+00:00,TornCurtain1991,677,2012-02-08 15:30:41+00:00,"A note: Stepan #Bandera, DOB 01011909, was lea...",1,2,0,0,"['Bandera', 'Ukraine']",en,NaN,NaN,278,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60128,1609034657231675393,2022-12-31 03:52:11+00:00,XgkXgk,2559,2014-07-16 02:00:17+00:00,#UkraineRussiaWar️ \n#Russia-#Ukraine #war: #P...,0,0,1,0,"['UkraineRussiaWar️', 'Russia', 'Ukraine', 'wa...",en,NaN,NaN,189,9,0,0
60129,1609032640664838145,2022-12-31 03:44:10+00:00,MarkKennedy721,2250,2013-06-27 19:26:17+00:00,Living it up in #Venezuela while their gov't b...,1,0,0,0,"['Venezuela', 'RussiaUkraineWar', 'Ukraine️', ...",en,NaN,NaN,190,6,0,0
60130,1609025141333438465,2022-12-31 03:14:22+00:00,Antiwarcom,69667,2008-07-20 16:05:20+00:00,Alexander Mercouris Comments on Ray McGovern’s...,89,47,2,1,"['Russia', 'Ukraine', 'NATO', 'Lavrov', 'Ukrai...",en,[Photo(previewUrl='https://pbs.twimg.com/media...,NaN,225,7,0,1
60131,1609020670486405121,2022-12-31 02:56:36+00:00,knossavage,299,2011-06-28 03:26:46+00:00,Nice overview of the capabilities of the Bradl...,1,1,0,0,"['Ukraine️', 'Ukrainian', 'UkraineRussiaWar️',...",en,NaN,NaN,217,8,0,0


Steps taken:
- Converting emojis to text
- We decide to remove all the mentions and hashtagged words, as these will be analysed separately
- Remove Links, as these don't contribute to SA
- Conducting the SA on our preprocessed data

Creating a new column so that we can see the adjusted tweet and original versiom

In [6]:
df.insert(loc=6,
          column='Adjusted Tweet',
          value=df['rendered_content'])

Converting emojis to text

In [7]:
import emot
import emoji

def demote(text):
    text = emoji.demojize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(demote)

ModuleNotFoundError: No module named 'emot'

Removing mentions and hashtagged words

In [8]:
import re

def remove_mentions_hashtags(text):
    text = re.sub("@[A-Za-z0-9_]+","", text)
    text = re.sub("#[A-Za-z0-9_]+","", text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_mentions_hashtags)

Removing links

In [9]:
import re

def remove_links(text):
    text = re.sub('http://\S+|https://\S+', '', text)
    text = re.sub('http[s]?://\S+', '', text)
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r'bit.ly/\S+', '', text) # remove bitly links
    text = text.strip('[link]') # remove [links]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_links)

Insert Polarity Score Column

In [10]:
df.insert(loc=7,
          column='Polarity Score',
          value=df['Adjusted Tweet'])

Sentiment Analysis using NLTK's VADER

In [11]:
from nltk.sentiment import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

def sentiment_analysis(text):  
    text = sia.polarity_scores(text)
    return text

df['Polarity Score'] = df['Polarity Score'].apply(sentiment_analysis)
    

LookupError: 
**********************************************************************
  Resource [93mvader_lexicon[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('vader_lexicon')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93msentiment/vader_lexicon.zip/vader_lexicon/vader_lexicon.txt[0m

  Searched in:
    - 'C:\\Users\\ekmho/nltk_data'
    - 'C:\\Users\\ekmho\\anaconda3\\nltk_data'
    - 'C:\\Users\\ekmho\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\ekmho\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\ekmho\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
    - ''
**********************************************************************


Creating columns for:
- Negative Score
- Neutral Score
- Positive Score
- Compound Score [-1,1]

In [ ]:
df.insert(loc=8,
          column='Negative Score',
          value=df['Polarity Score'])

df.insert(loc=9,
          column='Neutral Score',
          value=df['Polarity Score'])

df.insert(loc=10,
          column='Positive Score',
          value=df['Polarity Score'])

df.insert(loc=11,
          column='Compound Score',
          value=df['Polarity Score'])

In [ ]:
def negative_score(text):
    text = text['neg']
    return text

df['Negative Score'] = df['Negative Score'].apply(negative_score)

In [ ]:
def neutral_score(text):
    text = text['neu']
    return text

df['Neutral Score'] = df['Neutral Score'].apply(neutral_score)

In [ ]:
def positive_score(text):
    text = text['pos']
    return text

df['Positive Score'] = df['Positive Score'].apply(positive_score)

In [ ]:
def compound_score(text):
    text = text['compound']
    return text

df['Compound Score'] = df['Compound Score'].apply(compound_score)

In [ ]:
df.head()

Sentiment Analysis using TextBlob

In [12]:
df.insert(loc=12,
          column='Polarity Score_textblob',
          value=df['Adjusted Tweet'])

In [13]:
df.insert(loc=13,
          column='Subjectivity Score_textblob',
          value=df['Adjusted Tweet'])

In [14]:
from textblob import TextBlob

#Create a function to get the subjectivity
def getSubjectivity(text):
    return TextBlob(text).sentiment.subjectivity

#Create a function to get the polarity
def getPolarity(text):
    return TextBlob(text).sentiment.polarity

df['Polarity Score_textblob'] = df['Polarity Score_textblob'].apply(getPolarity)
df['Subjectivity Score_textblob'] = df['Subjectivity Score_textblob'].apply(getSubjectivity)


In [15]:
df.head()

,id,date,user,user_followers,user_created,rendered_content,Adjusted Tweet,Polarity Score,likes,retweets,replies,quoteCount,Polarity Score_textblob,Subjectivity Score_textblob,hashtags,lang,media,mentionedUsers
0,1477420789863436289,2022-01-01 23:25:40+00:00,anno1540,8838,2014-06-12 17:05:22+00:00,"Lithuania will never abandon Ukraine, voluntee...","Lithuania will never abandon Ukraine, voluntee...","Lithuania will never abandon Ukraine, voluntee...",5,1,0,0,0.000000,0.000000,"['Lithuania', 'Ukraine']",en,NaN,NaN
1,1477414596424220679,2022-01-01 23:01:03+00:00,weather_odessa,119,2019-07-10 08:34:22+00:00,#odessa #odesa #ukraine #одесса\nNow: 4.2°C\nT...,#одесса\nNow: 4.2°C\nToday's Min: 4.2°C at ...,#одесса\nNow: 4.2°C\nToday's Min: 4.2°C at ...,0,0,0,0,0.000000,0.000000,"['odessa', 'odesa', 'ukraine', 'одесса']",en,NaN,NaN
2,1477414332376010752,2022-01-01 23:00:00+00:00,AlArabiya_Eng,927174,2009-02-28 08:31:32+00:00,After tough talk between Presidents Joe Biden ...,After tough talk between Presidents Joe Biden ...,After tough talk between Presidents Joe Biden ...,4,0,3,0,-0.194444,0.666667,"['Russia', 'Ukraine']",en,NaN,NaN
3,1477409748572151809,2022-01-01 22:41:47+00:00,beatravelling,6329,2014-02-28 21:25:33+00:00,The beach can be nice in the fall too 😊🇺🇦\n\n#...,The beach can be nice in the fall too 😊🇺🇦\n\n ...,The beach can be nice in the fall too 😊🇺🇦\n\n ...,0,0,0,0,0.600000,1.000000,"['lanzheron', 'langeron', 'beach', 'odessa', '...",en,NaN,NaN
4,1477409332820119552,2022-01-01 22:40:08+00:00,TornCurtain1991,677,2012-02-08 15:30:41+00:00,"A note: Stepan #Bandera, DOB 01011909, was lea...","A note: Stepan , DOB 01011909, was leader of O...","A note: Stepan , DOB 01011909, was leader of O...",1,2,0,0,-0.100000,0.033333,"['Bandera', 'Ukraine']",en,NaN,NaN


## Further manipulating the tweet

Steps taken:
- Lowercase
- Punctuation
- Tokenization
- Stopword filtering
- Stemming

Changing all text to lowercase

In [16]:
def lowercase(text):    
    text = text.lower()
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(lowercase)

Removing all Punctuation

In [17]:
import string

def punctuation_remove(text):
    text = "".join([char for char in text if char not in string.punctuation])
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(punctuation_remove)

Tokenizing

In [18]:
from nltk import word_tokenize

def tokenize(text):
    text = word_tokenize(text)
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(tokenize)

Stopword Filtering

In [19]:
from nltk.corpus import stopwords

def remove_stopwords(text):
    stop_words = stopwords.words('english')
    text = [word for word in text if word not in stop_words]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(remove_stopwords)

Stemming

In [20]:
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()

def stem(text):
    stemmed = [porter.stem(word) for word in text]
    return text

df['Adjusted Tweet'] = df['Adjusted Tweet'].apply(stem)

## Months since creation of account relative to tweet

In [21]:
#Finding date of account creation in months
df.insert(loc=5,
          column='Date of Creation in months',
          value=df['user_created'])

In [22]:
from datetime import *

#returning the months of account creation
def account_creation(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    year = str(text)[0:4]
    month = str(text)[5:7]
    total_months = (int(year)*12)+(int(month))
    return (total_months)

df['Date of Creation in months'] = df['Date of Creation in months'].apply(account_creation)

In [23]:
#Finding date of tweet in months
df.insert(loc=6,
          column='Date of Tweet in Months',
          value=df['date'])

In [24]:
from datetime import *

#return the year of tweet
df['Date of Tweet in Months'] = df['Date of Tweet in Months'].apply(account_creation)

In [25]:
#Calculating Months since creation of account relative to tweet
df.insert(loc=7,
          column='Months Since Creation of Account',
          value= (df['Date of Tweet in Months']-df['Date of Creation in months']))

## Time of Day

In [26]:
#create a column for hours:
df.insert(loc=2,
          column='hour of tweet',
          value=df['date'])

In [27]:
from datetime import *

#return the hour of the tweet
def hour(text):
    text = datetime.strptime(text, "%Y-%m-%d %H:%M:%S+00:00")
    hour = str(text.time())[0:2]
    return int(hour)

df['hour of tweet'] = df['hour of tweet'].apply(hour)

In [28]:
#insert column for the time of day
df.insert(loc=3,
          column='time of day',
          value=df['hour of tweet'])

In [29]:
#calculating the time of day
def time_of_day(text):  
    if ((text > 4) and (text < 8 )):
        return 'Early Morning'
    elif ((text > 8) and (text < 12 )):
        return 'Morning'
    elif ((text > 12) and (text < 16 )):
        return 'Noon'
    elif ((text > 16) and (text < 20 )):
        return 'Eve'
    elif ((text > 20) and (text < 24 )):
        return 'Night'
    elif ((text > 0) and (text < 4 )):
        return 'Late Night'
    
df['time of day'] = df['time of day'].apply(time_of_day)


In [30]:
#Creating counts using one hot encoding

#Early Morning Count
df.insert(loc=4,
          column='Early Morning Count',
          value=df['time of day'])

def early_morning_count(text):
    if text == 'Early Morning':
        return 1
    else:
        return 0

df['Early Morning Count'] = df['Early Morning Count'].apply(early_morning_count)

#Morning Count
df.insert(loc=5,
          column='Morning Count',
          value=df['time of day'])

def morning_count(text):
    if text == 'Morning':
        return 1
    else:
        return 0

df['Morning Count'] = df['Morning Count'].apply(morning_count)

#Noon count
df.insert(loc=6,
          column='Noon Count',
          value=df['time of day'])

def noon_count(text):
    if text == 'Noon':
        return 1
    else:
        return 0

df['Noon Count'] = df['Noon Count'].apply(noon_count)

#Eve count
df.insert(loc=7,
          column='Eve Count',
          value=df['time of day'])

def eve_count(text):
    if text == 'Eve':
        return 1
    else:
        return 0

df['Eve Count'] = df['Eve Count'].apply(eve_count)

#Night count
df.insert(loc=8,
          column='Night Count',
          value=df['time of day'])

def night_count(text):
    if text == 'Night':
        return 1
    else:
        return 0

df['Night Count'] = df['Night Count'].apply(night_count)

#Late Night count
df.insert(loc=9,
          column='Late Night Count',
          value=df['time of day'])

def late_night_count(text):
    if text == 'Late Night':
        return 1
    else:
        return 0

df['Late Night Count'] = df['Late Night Count'].apply(late_night_count)

## Video, GIF and Photo Count

Photo Count

In [31]:
#Creating a photo count column...
df.insert(loc=23,
          column='Photo Count',
          value=df['media'])

In [32]:
#Counting number of Photos in media column
#No need to tokenize
def photo_count(text):
    text = str(text)
    text = text.count('Photo')
    return text

df['Photo Count'] = df['Photo Count'].apply(photo_count)


Video Count

In [33]:
#Creating a video count column...
df.insert(loc=24,
          column='Video Count',
          value=df['media'])

In [34]:
#We need to tokenize the media column so that we can count how many videos there are...
from nltk import word_tokenize

def tokenize(text):
    text = str(text)
    text = word_tokenize(text)
    return text

df['Video Count'] = df['Video Count'].apply(tokenize)

In [35]:
#Counting number of Videos in media column
def video_count(text):
    text = text.count('Video')
    return text

df['Video Count'] = df['Video Count'].apply(video_count)

Gif Count

In [36]:
#Creating a GIF count column...
df.insert(loc=25,
          column='GIF Count',
          value=df['media'])

In [37]:
#We need to tokenize the media column so that we can count how many GIFs there are...
from nltk import word_tokenize

def tokenize(text):
    text = str(text)
    text = word_tokenize(text)
    return text

df['GIF Count'] = df['GIF Count'].apply(tokenize)

In [38]:
#Counting number of GIFs in media column
def gif_count(text):
    text = text.count('Gif')
    return text

df['GIF Count'] = df['GIF Count'].apply(gif_count)

## Creating our final Dataframe

Steps taken:
- Drop: media, date, user, polarity score, user_created, Date of Creation in months, Date of Tweet in Months, rendered_content.	

In [39]:
df.drop(['media','date','user','Polarity Score','user_created','Date of Creation in months','Date of Tweet in Months','rendered_content'], axis=1)

,id,hour of tweet,time of day,Early Morning Count,Morning Count,Noon Count,Eve Count,Night Count,Late Night Count,user_followers,...,replies,quoteCount,Photo Count,Video Count,GIF Count,Polarity Score_textblob,Subjectivity Score_textblob,hashtags,lang,mentionedUsers
0,1477420789863436289,23,Night,0,0,0,0,1,0,8838,...,0,0,0,0,0,0.000000,0.000000,"['Lithuania', 'Ukraine']",en,NaN
1,1477414596424220679,23,Night,0,0,0,0,1,0,119,...,0,0,0,0,0,0.000000,0.000000,"['odessa', 'odesa', 'ukraine', 'одесса']",en,NaN
2,1477414332376010752,23,Night,0,0,0,0,1,0,927174,...,3,0,0,0,0,-0.194444,0.666667,"['Russia', 'Ukraine']",en,NaN
3,1477409748572151809,22,Night,0,0,0,0,1,0,6329,...,0,0,0,0,0,0.600000,1.000000,"['lanzheron', 'langeron', 'beach', 'odessa', '...",en,NaN
4,1477409332820119552,22,Night,0,0,0,0,1,0,677,...,0,0,0,0,0,-0.100000,0.033333,"['Bandera', 'Ukraine']",en,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60128,1609034657231675393,3,Late Night,0,0,0,0,0,1,2559,...,1,0,0,0,0,0.000000,0.000000,"['UkraineRussiaWar️', 'Russia', 'Ukraine', 'wa...",en,NaN
60129,1609032640664838145,3,Late Night,0,0,0,0,0,1,2250,...,0,0,0,0,0,0.000000,0.000000,"['Venezuela', 'RussiaUkraineWar', 'Ukraine️', ...",en,NaN
60130,1609025141333438465,3,Late Night,0,0,0,0,0,1,69667,...,2,1,2,0,0,0.000000,0.000000,"['Russia', 'Ukraine', 'NATO', 'Lavrov', 'Ukrai...",en,NaN
60131,1609020670486405121,2,Late Night,0,0,0,0,0,1,299,...,0,0,0,0,0,0.600000,1.000000,"['Ukraine️', 'Ukrainian', 'UkraineRussiaWar️',...",en,NaN
